# Convert .SNPY files to my format

Originally written in python 2.7, this script runs also in python 3.7.3 Anaconda.

There are two options in this notebook:

    1. Convert ".snpy" files that are located in a single folder.
    
    2. Convert ".snpy" files that are located in a multiple folders.

Create text files of:
    - (phase, absolute mag, error mag) and
    - (phase, apparent mag, error mag)

- Based from 06_Canopy_AbsAppMag_Phase_ForEachBand_Main.ipynb
- Useful to convert SNooPy LC files to my format for high redshift.

In [1]:
from snpy import *
# from numpy import * 
import numpy as np
from matplotlib import pyplot as plt
import datetime # Get the current date and time
import json
import glob # To read file names in a directory
# To read arguments in command line
# Used in the ".py" version of this notebook.
import sys 

# %pylab qt 
# For CANOPY python: to show the plots in a separated Windows instead of inline.
# If used, then I don't have to put the instruction "plt.close()" at
# the end of the plot because it will show the figure and instantaneously 
# close the windows too.

#--------------------------------------------------------60
code_created_by = 'Arturo_Avelino'
# On date: 2019.02.28 (yyyy.mm.dd)
code_name = 'convert_snpy2myformat.ipynb'
code_version = '0.1.2'
code_last_update = '2019.09.02'
#--------------------------------------------------------60

In [ ]:
##############################################################################80

# User

In [2]:
# Directory where the ".snpy" files are located:

list_file='list_SNe_snpy.txt'

dir_data = '/Users/arturo/Documents/Research/Workplace/raisin/raisin1/\
2019_08_05__v0_1_1/fits/EBV/Mangle/All_tbmaxFix/'

# Change where the .snpy files are located:
os.chdir(dir_data)

#--------------------------------------------------------60
# Read the list with the names of the .snpy files

list_sne = np.genfromtxt(dir_data+list_file, dtype=['S22'])

print('# %s SNe in this file.'%len(list_sne))
print(list_sne[:3])

#--------------------------------------------------------60
# Directory to save the output

dir_save_output = dir_data+'MyFormat/data/'

# Dir to save the 'ListSNe_AppMag_.txt' list:
dir_save_output_2 = dir_data+'MyFormat/fit/'

#- If the subdirectory does not exist then create it
if not os.path.exists(dir_save_output): os.makedirs(dir_save_output)
if not os.path.exists(dir_save_output_2): os.makedirs(dir_save_output_2)

# 17 SNe in this file.
[(b'PScA470110_Fit.snpy',) (b'PScA470240_Fit.snpy',)
 (b'PScB480794_Fit.snpy',)]


### Settings

In [3]:
# Settings

band = 'f160w'  # Observer-frame band

stretch_par = 'dm15' # options ( dm15 , stretch )

err_zhelio = 0.005
err_zcmb = err_zhelio
err_kcorr = 0.001

#-------------------------------

# The value HoFix = 73.24 is what we use in the low-z paper.
HoFix = 73.24 # Hubble constant (km/(s Mpc))
# HoFix = 72.78  # TEMPORAL: value reported by Dhawan et al 2017.

OmMFix = 0.28
OmLFix = 0.72
wFix = -1.0

cc = 299792.458  # Speed of light (km/s)

# Print some marks to debug the code?
debug = False

#--------------------------------------------------------60

#-- Consider the light curves with -at least- the following number of data:
MinNumberOfDataInLCs=1

#-- Consider the LC that contains data in the following range -at least- only:
# phase_min = 3
# phase_max = 10

phase_min = 50
phase_max = -15


In [4]:
##############################################################################80

--------

# Automatic

### Uploading the SED of Hsiao to compute the extinction

In [5]:
Ia_w, Ia_f = kcorr.get_SED(0,'H3')

### Cosmology theory for $\mu(z)$

In [6]:
from scipy.integrate import quad as intquad

# Inverse of the dimensionless Hubble parameter
def InvEHubblePar(z, OmM, wde):
    "Dimensionless Hubble parameter"
    InvEHubbleParInt = 1.0/(np.sqrt(OmM*(1.0+z)**3.0 + (1.0-OmM)*(1.+z)**(3.*(1.+wde))))
    return InvEHubbleParInt

# ---- The luminosity distance ----
def LumDistance(z, OmM, wde, Ho):
    "Luminosity distance"
    LumDistanceVecInt = 0.
    LumDistanceVecInt = cc*(1.+z)*intquad(InvEHubblePar, 0., z, args=(OmM, wde))[0]/Ho 
    return LumDistanceVecInt


# ---- Distance modulus ----
def DistanceMu(z, OmM, wde, Ho):
    "Distance modulus"     
    DistanceMuInt = 5.0*np.log10(LumDistance(z, OmM, wde, Ho)) + 25.0
    return DistanceMuInt

#--------------------------------------------------

ztest1 = 0.1
print(DistanceMu(ztest1, OmMFix, wFix, HoFix))
# 38.2572823409
# 38.22020314489771

38.22020314489771


---

### Definition of the main function

In [7]:
#-----------------------------------------------------------------------------80
def snpy2myformat(snfile, band,
        zcmb = -1, err_zcmb = 0.005,
        err_zhelio = 0.005, err_kcorr = 0.001,
        stretch_par = 'dm15',
        HoFix = 73.24, OmMFix = 0.28, OmLFix = 0.72, wFix = -1.0,
        MinNumberOfDataInLCs = 1, 
        suffix_extension = '_myformat.txt',
        dir_save_output = '',
        debug = False):

    """
    Function to create a text file with the format required to use it
    as input in my script to fit the light curve (LC) data with my template and
    Gaussian-process (GP) methods.

    The input file is the .snpy file generated by SNooPy when fitting the
    LC data.
    
    - stretch_par (str): stretch parameter. Options ('dm15', 'stretch')
    - suffix_extension (str): suffix and extension to use to write in the
        output filename.
    """

    import datetime # Get the current date and time
    cc = 299792.458  # Speed of light (km/s)

    # Flag to indicate that the conversion was sucessful.
    # flag=1: fail, flag=1: sucess. 
    flag = 1 # reset
    
    if debug: print('#       %s'%snfile)
        
    # Upload the LC data
    s = get_sn(snfile)
    if debug: print('# %s .snpy file read.'%s.name)
    

    # Define zcmb:
    if zcmb != -1: zcmb_int = zcmb
    else: zcmb_int = s.zcmb

    # s.summary()

    # Check if the LC file has data in band I'm interested, and if
    # there are enough data in that band.
    if (band in s.data.keys() and
        len(s.data[band].MJD) >= MinNumberOfDataInLCs):
        
        try:

            #------- The extinction A_lambda --------
            R_F = fset[band].R(wave=Ia_w, flux=Ia_f, Rv=3.1)
            Alamb = R_F * s.EBVgal
            e_Alamb = R_F * s.e_EBVgal

            #------- LCDM distance modulus
            mu_LCDM = DistanceMu(zcmb_int, OmMFix, wFix, HoFix)

            #--------------------------------------------------------
            # Save the photometry to a numpy array.
            # Each row in the 'data_band_np' numpy array will have the data:
            #(Phase(TBmax), Abs mag, Err_Absmag, App mag, Err_appmag, kcorr, err_kcorr)

            count_AllNumDataInLC = 0
            count_NumDataInLC = 0

            # Loop over MJD in a given band
            for i2 in range(len(s.data[band].MJD)):

                # ---- numpy array: begins ---->>
                data_np = np.array([

                1, # just a dummy value.

                # phase:
                (s.data[band].MJD[i2]-s.Tmax)/((1+s.z)*s.ks_s),

                # Abs mag: kcorrected, MWdust corrected, and distance
                # modulus subtracted:
                s.data[band].mag[i2] - s.ks[band][i2] - Alamb - mu_LCDM,

                # Total uncertainty for abs mag =
                # = (e_mag^2 + e_Alamb^2 + e_kcorr^2)^(-1/2)
                np.sqrt((s.data[band].e_mag[i2])**2 + e_Alamb**2 +
                err_kcorr**2 ),

                # Apparent magnitude: MWdust corrected
                s.data[band].mag[i2] - s.ks[band][i2] - Alamb,

                # Apparent magnitude error
                np.sqrt((s.data[band].e_mag[i2])**2 + e_Alamb**2),

                # k-corr
                s.ks[band][i2],

                # k-corr error
                err_kcorr  ])
                # <<----- numpy array: ends ---

                # print "%s index"%i
                if i2 == 0:
                    data_band_np = data_np
                elif i2 > 0:
                    data_band_np = np.vstack((data_band_np, data_np))

                count_AllNumDataInLC += 1
                count_NumDataInLC += 1

            #------------------------

            # Save to a new array, 'data_all_np', all the -good- photometry
            # in a given band:

            CountNumSimilarBands=1

            if CountNumSimilarBands == 1:
                data_all_np = data_band_np
            else:
                data_all_np =  np.vstack((data_all_np , data_band_np))

            #--------------------------------------------------------
            # Sort the data based on the phase values.

            try:
                # Sort the array in increasing order of phase.
                data_all_2_np = np.sort(data_all_np.view('f8,f8,f8,f8,f8,f8,f8,f8'),
                                        order=['f1'], axis=0).view(np.float)

                if debug: print("# Step. All %s data sorted; \
            done."%len(data_all_2_np))

            # When there is just one datum in the LC:
            except:
                data_all_2_np = data_all_np
                if debug: print("# Step. Only one datum in the photometry; done.")

            #--------------------------------------------------------
            # Write the photmetry with my format.

            # Suffix to add to the original file name:
            extension_myformat = suffix_extension
            # old. extension_myformat = '_%s_CfA_0.txt'%band

            # Output name:
            output_name = snfile[0:-5]+extension_myformat

            SN_txtfile = open(dir_save_output+output_name, 'w')

            #----------- Write header -----------

            SN_txtfile.write("#     {0} \n".format(s.name))
            SN_txtfile.write("# Source '.snpy' file used to create this file: %s\n"%snfile)
            SN_txtfile.write("# Located at:\n")
            SN_txtfile.write("# %s\n"%dir_data)
            text_line = '#'+'-'*57 + '60\n'
            SN_txtfile.write(text_line)

            SN_txtfile.write("%-13.6f  %.6f  %.6f  %.6f  -1       -1  -1 # \
(zcmb, err_zcmb, zhelio, err_zhelio) \n"%(
                    zcmb_int, err_zcmb, s.z, err_zhelio))
            
            if debug: print("# Step. Redshifts written.")
    
            # Stretch parameter EBV used dm15, and EBV2 uses the stretch:
            if stretch_par == 'dm15':
                SN_txtfile.write("%-13.6f  %.6f  %.6f   -1       -1       -1  -1 # \
(dm15, err_dm15, stretch) \n"%(
                    s.dm15, s.e_dm15, s.ks_s))
                if debug: print("# dm15 parameter = %-13.6f"%s.dm15)
            
            elif stretch_par == 'stretch':
                SN_txtfile.write("%-13.6f  %.6f  %.6f   -1       -1       -1  -1 # \
(stretch, err_stretch, stretch) \n"%(
                    s.st, s.e_st, s.ks_s))
                if debug: print("# Stretch parameter = %-13.6f"%s.st)
                    
                    
            if debug: print("# Step. dm15 and/or stretch parameters written.")

            SN_txtfile.write("%-13.6f  %.6f  %.6f  -1       -1       -1  -1 # \
(mu_Snoopy, err_mu_Snoopy, mu_LCDM) \n"%(
                    s.DM, s.e_DM, mu_LCDM))

            SN_txtfile.write("%-13.6f  %.6f  %.6f  %.6f  %.6f -1  -1 # \
(E(B-V)_MW, err_E(B-V)_MW, Alamb, err_Alamb, R_F) \n"%(
                    s.EBVgal, s.e_EBVgal, Alamb, e_Alamb, R_F))

            SN_txtfile.write("%-13.6f  %.6f  -1        -1        -1       -1  -1 # \
(E(B-V)_host, err_E(B-V)_host) \n"%(
                    s.EBVhost, s.e_EBVhost))

            SN_txtfile.write("%-13.6f  %.6f  -1        -1        -1       -1  -1 # \
(T_Bmax, err_T_Bmax) \n"%(s.Tmax, s.e_Tmax))

            SN_txtfile.write("-1  -1  -1  -1  -1  -1  -1 # Free slot \n")
            SN_txtfile.write("-1  -1  -1  -1  -1  -1  -1 # Free slot \n")

            SN_txtfile.write("# Assuming flat LCDM with (Om={0}, OL={1}, w={2}, \
Ho={3}) \n".format(OmMFix, OmLFix, wFix, HoFix))

            #------info about the creation of the file---------------

            # Define the header text
            now = datetime.datetime.now() # Read the time and date right now
            text_timenow = now.strftime("%Y.%m.%d (yyyy.mm.dd); %H:%M hrs (CDT).")
            text_Date   = '# On date: %s\n'%text_timenow
            text_Author = '# Data table created by: %s.\n'%code_created_by
            text_script = '# Script used: %s (version %s | last update: %s)\n'%(
                code_name, code_version, code_last_update)

            # Write the header text
            SN_txtfile.write(text_line);
            SN_txtfile.write(text_Author); SN_txtfile.write(text_Date);
            SN_txtfile.write(text_script);
            SN_txtfile.write(text_line);

            #-----------------------------------------

            SN_txtfile.write("#      %s \n"%band)
            SN_txtfile.write('#Phase(TBmax) Abs mag  Err_Absmag  App mag  \
Err_appmag   kcorr    err_kcorr  \n')

            #----------- Write photometry -----------

            for i3 in range(len(data_all_2_np)):
                CommentText = ''
                SN_txtfile.write("%s%-9.5f   %.6f  %.6f  %.6f  %.6f  %10.6f  %.6f\n"%(
                        CommentText,
                        data_all_2_np[i3][1], data_all_2_np[i3][2], data_all_2_np[i3][3],
                        data_all_2_np[i3][4], data_all_2_np[i3][5], data_all_2_np[i3][6],
                        data_all_2_np[i3][7]) )

            SN_txtfile.close();
            
            flag = 0 # Flag to indicate that the conversion was sucessful.
            out_message = print('# %s file created.'%output_name)
            
        except:
            flag = 1 # Flag to indicate that the conversion was failed.
            out_message = print('# %s:  Data in %s-band but it was not fitted by SNooPy!'%(snfile, band))

    #--------------------------------------------------------
    else:
        flag = 1 # Flag to indicate that the conversion was failed.
        out_message = print('# %s:  No data in band %s!'%(snfile, band))

    return flag, out_message
#-----------------------------------------------------------------------------80

---

# Loop: for one case (ok)

In [ ]:
# Create the the list 'ListSNe_AppMag_.txt':
textfile_1 = open(dir_save_output_2+'ListSNe_AppMag_.txt','w')
textfile_1.write('#   List of LC files with my format\n')

count_sne_within_subcases = 0

# Loop over ".snpy" files
for i1 in range(len(list_sne)):

    # snfile = str(list_sne[i1][0])  # Python 2.7
    snfile = list_sne[i1][0].decode('UTF-8')  # Python 3.7
    # print('     ', snfile, ": I'll try to fit it.")

    flag, verbose = snpy2myformat(snfile, band, 
        # zcmb = zcmb, 
        err_zcmb = err_zcmb, err_zhelio = err_zhelio, 
        err_kcorr = err_kcorr,
        stretch_par = stretch_par,
        HoFix = HoFix, OmMFix = OmMFix, OmLFix = OmLFix, wFix = wFix, 
        MinNumberOfDataInLCs = MinNumberOfDataInLCs, 
        suffix_extension = '_%s_CfA_0.txt'%band,
        dir_save_output = dir_save_output,
        debug=False)

    print('')

    # Write down the file name of the SNe only if it was converted
    # to my format sucessfully:
    if flag==0:
        textfile_1.write('%s    0\n'%(snfile[:-5]+'_%s_CfA_0.txt'%band))
        count_sne_within_subcases += 1

textfile_1.close()
print("# %s '.snpy' files converted."%count_sne_within_subcases)


In [ ]:
##############################################################################80

----

# Loop: for several cases (OK)

- loop over cases subdirectories:
    - define the stretch parameter depending on EBV or EBV2.
    - define band (actually fixed).
    - the output file is defined automatically.
    
- Write a text file with the names of the files converted and with a column 
of zeros to mask SNe names later on if needed.

- WATCH OUT: For RAISIN-1 append the suffix "CSP", and for RAISIN-2 the suffix "CfA"

### User

In [11]:
# Read the the file listing all the subcases

file_list_cases = 'List_dir_cases.txt'
dir_list_cases  = '/Users/arturo/Documents/Research/Workplace/raisin/raisin1/\
2019_08_05__v0_1_1/fits/'

list_subdirs_cases = np.genfromtxt(dir_list_cases+file_list_cases,dtype=['S50'])
print('# %s sudirectories listed in this file.'%len(list_subdirs_cases))

# 9 sudirectories listed in this file.


In [12]:
# Root directory. This is the main directory where all the
# subcases folders are located.

dir_root = '/Users/arturo/Documents/Research/Workplace/raisin/raisin1/\
2019_08_05__v0_1_1/fits/'

In [13]:
list_subdirs_cases

array([(b'EBV/Mangle/All_tbmaxFix/',),
       (b'EBV/Mangle/Optical_g_tbmaxFix/',),
       (b'EBV/NoMangle/All_tbmaxFix/',),
       (b'EBV/NoMangle/Optical_g_tbmaxFix/',),
       (b'EBV2/Mangle/Optical_g_tbmaxFix/',),
       (b'EBV2/Mangle/Optical_NIR_g_tbmaxFix/',),
       (b'EBV2/NoMangle/All_tbmaxFix/',),
       (b'EBV2/NoMangle/Optical_g_tbmaxFix/',),
       (b'EBV2/NoMangle/Optical_NIR_g_tbmaxFix/',)], dtype=[('f0', 'S50')])

#### Loop

In [18]:
# Append the following suffixes:
# RAISIN-1: 'CSP'
# RAISIN-2: 'CfA'

suffix_output = '_%s_CSP_0.txt'%band

#-----------------------------------------------------------------------------80
# Filename of .snpy list. This name is the -same- for all subcases.
list_file='list_SNe_snpy.txt'

count_subcases = 0

# Loop over cases:
for i2 in range(len(list_subdirs_cases)):
    
    # Define the directory of the subcase:
    dir_subcase = list_subdirs_cases[i2][0].decode('UTF-8')
    
    # Define the full path to the subcase:
    dir_data = dir_root + dir_subcase
    
    # Directory to save the output
    dir_save_output   = dir_data+'MyFormat/data/'
    dir_save_output_2 = dir_data+'MyFormat/fit/'
    #- If the subdirectory does not exist then create it
    if not os.path.exists(dir_save_output): os.makedirs(dir_save_output)
    if not os.path.exists(dir_save_output_2): os.makedirs(dir_save_output_2)
    
    # Change to the subdirectory:
    os.chdir(dir_data)
    
    # Read the list with the names of the .snpy files
    list_sne = np.genfromtxt(dir_data+list_file, dtype=['S22'])
    
    print("\n#--------- %s ----------\n"%dir_subcase)
    
    # Define the snoopy model (i.e., EBV or EBV2) by gathering this 
    # info from the  subdirectory's name: 
    snoopy_model = dir_subcase[:4]
    
    # Define the stretch parameter to use based on the snoopy model:
    if   snoopy_model=='EBV/': stretch_par = 'dm15'
    elif snoopy_model=='EBV2': stretch_par = 'stretch'
    
    # print (stretch_par)
    ,
    #--------------------------------------------------------
    # Loop over ".snpy" files
    
    textfile_1 = open(dir_save_output_2+'ListSNe_AppMag_.txt','w')
    textfile_1.write('#   List of LC files with my format\n')
    
    count_sne_within_subcases = 0
    
    # Loop over ".snpy" files
    for i1 in range(len(list_sne)):
        
        # snfile = str(list_sne[i1][0])  # Python 2.7
        snfile = list_sne[i1][0].decode('UTF-8')  # Python 3.7
        # print('     ', snfile, ": I'll try to fit it.")

        flag, verbose = snpy2myformat(snfile, band, 
            # zcmb = zcmb, 
            err_zcmb = err_zcmb, err_zhelio = err_zhelio, 
            err_kcorr = err_kcorr,
            stretch_par = stretch_par,
            HoFix = HoFix, OmMFix = OmMFix, OmLFix = OmLFix, wFix = wFix, 
            MinNumberOfDataInLCs = MinNumberOfDataInLCs, 
            suffix_extension = suffix_output,
            dir_save_output = dir_save_output,
            debug=False)
        
        print('')
        
        # Write down the file name of the SNe only if it was converted
        # to my format sucessfully:
        if flag==0:
            textfile_1.write('%s    0\n'%(snfile[:-5]+suffix_output))
            count_sne_within_subcases += 1
    
    textfile_1.close()
    print("# %s '.snpy' files converted."%count_sne_within_subcases)
    
    #--------------------------------------------------------
    
    count_subcases += 1

print('\n#------------------------')
print('# %s subcases converted.'%count_subcases)


#--------- EBV/Mangle/All_tbmaxFix/ ----------

# PScA470110_Fit_f160w_CSP_0.txt file created.

# PScA470240_Fit_f160w_CSP_0.txt file created.

# PScB480794_Fit_f160w_CSP_0.txt file created.

# PScC490037_Fit_f160w_CSP_0.txt file created.

# PScE510457_Fit_f160w_CSP_0.txt file created.

# PScF520062_Fit_f160w_CSP_0.txt file created.

# PScF520107_Fit_f160w_CSP_0.txt file created.

# PScF520188_Fit_f160w_CSP_0.txt file created.

# PScG530251_Fit_f160w_CSP_0.txt file created.

# PScH540087_Fit_f160w_CSP_0.txt file created.

# PScH540118_Fit_f160w_CSP_0.txt file created.

# PScJ440005_Fit_f160w_CSP_0.txt file created.

# PScJ440236_Fit_f160w_CSP_0.txt file created.

# PScJ550202_Fit_f160w_CSP_0.txt file created.

# PScJ560027_Fit_f160w_CSP_0.txt file created.

# PScJ560054_Fit_f160w_CSP_0.txt file created.

# PScK450339_Fit_f160w_CSP_0.txt file created.

# 17 '.snpy' files converted.

#--------- EBV/Mangle/Optical_g_tbmaxFix/ ----------

# PScA470041_Fit.snpy:  Data in f160w-band but it 

/Users/arturo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: genfromtxt: Empty input file: "/Users/arturo/Documents/Research/Workplace/raisin/raisin1/2019_08_05__v0_1_1/fits/EBV2/Mangle/Optical_NIR_g_tbmaxFix/list_SNe_snpy.txt"


# PScF520062_Fit.snpy:  Data in f160w-band but it was not fitted by SNooPy!

# PScF520107_Fit_f160w_CSP_0.txt file created.

# PScF520188_Fit.snpy:  Data in f160w-band but it was not fitted by SNooPy!

# PScG530251_Fit_f160w_CSP_0.txt file created.

# PScH540087_Fit.snpy:  Data in f160w-band but it was not fitted by SNooPy!

# PScH540118_Fit_f160w_CSP_0.txt file created.

# PScJ440005_Fit.snpy:  Data in f160w-band but it was not fitted by SNooPy!

# PScJ440236_Fit_f160w_CSP_0.txt file created.

# PScJ550202_Fit_f160w_CSP_0.txt file created.

# PScJ560027_Fit_f160w_CSP_0.txt file created.

# PScJ560054_Fit_f160w_CSP_0.txt file created.

# PScK450082_Fit.snpy:  Data in f160w-band but it was not fitted by SNooPy!

# PScK450339_Fit_f160w_CSP_0.txt file created.

# 12 '.snpy' files converted.

#--------- EBV2/NoMangle/Optical_g_tbmaxFix/ ----------

# PScA470240_Fit.snpy:  Data in f160w-band but it was not fitted by SNooPy!

# PScB480464_Fit.snpy:  Data in f160w-band but it was not fitted b

/Users/arturo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: genfromtxt: Empty input file: "/Users/arturo/Documents/Research/Workplace/raisin/raisin1/2019_08_05__v0_1_1/fits/EBV2/NoMangle/Optical_NIR_g_tbmaxFix/list_SNe_snpy.txt"


In [19]:
textfile_1.close(); textfile_1.close(); textfile_1.close();